In [ ]:
"""
Project Title: Simulating electrolytic DNA chemistry on noisy quantum devices

Code is written by: Dowarah, Sasanka.
Affiliation: The University of Texas at Dallas

Description:
------------
The code is designed to produce the raw data and the plot of energy current in Allox dopd DNA as a function of time on a noiseless and noisy simulator.

Usage:
------
1. Dependencies:
    -  Python 3.x, NumPy, matplotlib, Qiskit, Qiskit-Nature, Qiskit-algorithms, Qiskit aer.

2. Running the Code:
    - Running the code uses Allox doped DNA fermionic oprator file from local directory saved in the same
    folder as the code. It saves the raw data in the simulator_data folder and the specified decay rate
    (gamma_in and gamma_out).

3. Configuration:
    - N/A

4. Testing:
    - N/A

5. Reproducibility:
    - This code is designed to produce consistent and reproducible results of the energy current in Allox doped DNA as a function of time for noiseless and noisy simulator.

""";

In [ ]:
import numpy as np
from qiskit import*
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.mappers import JordanWignerMapper, BravyiKitaevMapper
import os, qiskit_aer, pickle, itertools
#print(qiskit_aer.__version__)

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_algorithms import TimeEvolutionProblem
from qiskit.primitives import Estimator  
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

"""
The following imports the custom trotter class that includes ancilla qubits for Lindblad operators. 
It is a modified version of the original trotter.py file from qiskit_algorithms located at:
https://github.com/qiskit-community/qiskit-algorithms/blob/main/qiskit_algorithms/time_evolvers/trotterization/trotter_qrte.py
We modified the file to include ancilla qubits for the Lindblad operators and renamed it as trotter_for_open_quantum_systems.py
"""
import trotter_for_open_quantum_systems as trotter

large = 20
med = 20
small = 20
params = {"axes.titlesize": med, "axes.titlepad": med,"legend.fontsize": med, "axes.labelsize": large,"axes.titlesize": med, "xtick.labelsize": large, "ytick.labelsize": large, "figure.titlesize": med}
plt.rcParams["font.family"] = "Helvetica"
plt.rcParams["font.serif"] = ["Helvetica Neue"]
#plt.rcParams['text.usetex'] = True  # Use LaTeX in the Plots.
plt.rcParams.update(params)

# Loading the Fermionic Operator and Mapping into Qubit Space

In [ ]:
# Allox DNA in solution with four molecular orbitals (MOs).
with open("OPESMe_4MOs_mp2def2tzvp.pickle", "rb") as f:
    fermionic_hamiltonian = pickle.load(f)
# with open("4MO_TransitionState1.pickle", "rb") as f:
#     fermionic_hamiltonian = pickle.load(f)    
# with open("4MO_TransitionState2.pickle", "rb") as f:
#     fermionic_hamiltonian = pickle.load(f)  
# with open("4MO_Product.pickle", "rb") as f:
#     fermionic_hamiltonian = pickle.load(f)  


# Allox DNA in vacuum with four molecular orbitals (MOs).
# with open("new_4MO_reactant_withoutcharges.pickle", "rb") as f:
#     fermionic_hamiltonian = pickle.load(f)
# with open("new_4MO_product_without_charges.pickle", "rb") as f:
#     fermionic_hamiltonian = pickle.load(f)

# Converting the fermionic Hamiltonian to a qubit Hamiltonian.
jw_mapper = JordanWignerMapper()
qubit_hamiltonian_jw = jw_mapper.map(fermionic_hamiltonian)

L = len(qubit_hamiltonian_jw.paulis.to_labels()[0]) # Number of qubits.

print("Number of terms in the qubit Hamiltonian after JW = ", len(qubit_hamiltonian_jw))

In [ ]:
# Converting the fermionic Hamiltonian to a qubit Hamiltonian using the Jordan-Wigner mapping.
jw_mapper = JordanWignerMapper()
qubit_hamiltonian_jw = jw_mapper.map(fermionic_hamiltonian)
L = len(qubit_hamiltonian_jw.paulis.to_labels()[0]) # Number of qubits.
print("Number of terms in the qubit Hamiltonian after JW = ", len(qubit_hamiltonian_jw))

In [ ]:
# We need to truncate the qubit Hamiltonian to a few terms to run it on a quantum computer.
cutoff_threshold = 5.e-3 # in Hartree units.
qubit_hamiltonian_truncated = qubit_hamiltonian_jw.chop(cutoff_threshold).simplify()
# Removing the constant term.
#qubit_hamiltonian_truncated = SparsePauliOp(qubit_hamiltonian_truncated.paulis.to_labels()[1:], qubit_hamiltonian_truncated.coeffs[1:])
print("Number of terms in the truncated qubit Hamiltonian = ", len(qubit_hamiltonian_truncated))

# Finding the qubit corresponding to LUMO

In [ ]:
"""
In this section we will determine the qubit corresponding to the LUMO of the effective Hamiltonian.
"""

def lowest_half_filled_states(Hamiltonian):
     """
     This function returns the lowest energy half-filled states of the fermionic_hamiltonian_matrix. 
     """
     # Generate all possible half-filled states.
     zeros = [0] * (L//2)
     ones = [1] * (L//2)
     binary_list = zeros + ones
     unique_permutations = set(itertools.permutations(binary_list))
     half_filled_states_lst = ["".join(map(str, perm)) for perm in unique_permutations]

     energy_half_filled_states = []
     # Calculate the energy of each half-filled state.
     for half_filled_state in half_filled_states_lst:
          #half_filled_state = half_filled_state[::-1]
          energy_half_filled_states.append((half_filled_state, np.real(Statevector.from_label(half_filled_state).expectation_value(Hamiltonian))))
     sorted_energy_half_filled_states = sorted(energy_half_filled_states, key = lambda x: x[1])
     return sorted_energy_half_filled_states  

In [ ]:
number_of_lowest_half_filled_states_to_print = 4
print(f"First few lowest half-filled states of the Hamiltonian are: {lowest_half_filled_states(qubit_hamiltonian_truncated)[:number_of_lowest_half_filled_states_to_print]}")

## Setting the LUMO qubit

In [ ]:
# We count from right to left as done in Qiskit. The rightmost qubit is the 0th qubit.
LUMO_index = 7 # (5 or 1) Reactant state ground state 00010111 or 01110001 and excited state 00110101 or 01010011.
#LUMO_index = 0 # (0 or 4) Transition state 1 ground state 11110000 or 00001111 and excited state 11100001 or 00011110
#LUMO_index = 6 # (2 or 6) Transition state  2 ground state 00110011 and excited state 00010111 or 01110001
#LUMO_index = 6 # (6 or 2) Product state ground state 00110011 (unique) and excited state 01110001 or 00010111.

# Setting up the IBM Simulator (noiseless and noisy)

In [ ]:
# This is from the website: https://docs.quantum.ibm.com/verify/building_noise_models.
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer.noise import NoiseModel

# from qiskit_aer.primitives import EstimatorV2 as Estimator
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from qiskit_aer.primitives import SamplerV2 as Sampler
# from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService()

# Noisy backend.
# real_device_backend = service.backend("ibm_brisbane")
# noise_model = NoiseModel.from_backend(real_device_backend) # backend is the real device backend specified earlier.

# Noiseless backend.
from qiskit_aer import AerSimulator
real_backend_simulator = AerSimulator()
sampler = Sampler()

def local_gate_folding(circuit, folding_number):
    """ 
    This function performs local gate folding. It iterates over each gate in the circuit and 
    appends the inverse of the gate followed by the gate itself. If this is done once, the noise
    is amplified by a factor of 3. If this is done twice, the noise is amplified by a factor of 5.
    Parameters
    ----------
    circuit : QuantumCircuit
        The input circuit.
    folding_number : int
        The number of times the gate folding is to be performed.
    Returns
    -------
    QuantumCircuit
        The folded circuit.
        
    """     
    folded_circuit = QuantumCircuit(circuit.num_qubits)
    for instr, qargs, cargs in circuit:
        folded_circuit.append(instr, qargs, cargs)
        folded_circuit.barrier(qargs)
        for _ in range(folding_number):
            # Reset gate twice if the gate is a reset gate as the reset gate is not a unitary gate.
            if instr.name == "reset":
                folded_circuit.append(instr, qargs, cargs)
                folded_circuit.barrier(qargs)
                folded_circuit.append(instr, qargs, cargs)
                continue
          
            # Append inverse and original gate for other gates.
            if not instr.is_parameterized():
                inverse_instr = instr.inverse()
                folded_circuit.append(inverse_instr, qargs, cargs)
                folded_circuit.barrier(qargs)
                folded_circuit.append(instr, qargs, cargs)
                folded_circuit.barrier(qargs)
    return folded_circuit

In [ ]:
""" 
This section creates a custom noise model to run the noisy simulation. This is taken from
IBM website:https://docs.quantum.ibm.com/guides/build-noise-models
"""

from qiskit_aer.noise import (
    NoiseModel,QuantumError,ReadoutError,depolarizing_error,pauli_error,thermal_relaxation_error,
)

# p_reset = 0.02
# p_meas = 0.02
# p_gate1 = 0.02
 
# # QuantumError objects
# error_reset = pauli_error([("X", p_reset), ("I", 1 - p_reset)])
# error_meas = pauli_error([("X",p_meas), ("I", 1 - p_meas)])
# error_gate1 = pauli_error([("X",p_gate1), ("I", 1 - p_gate1)])
# error_gate2 = error_gate1.tensor(error_gate1)
 
# # Add errors to noise model
# noise_bit_flip = NoiseModel()
# noise_bit_flip.add_all_qubit_quantum_error(error_reset, "reset")
# noise_bit_flip.add_all_qubit_quantum_error(error_meas, "measure")
# noise_bit_flip.add_all_qubit_quantum_error(error_gate1, ["u1", "u2", "u3"])
# noise_bit_flip.add_all_qubit_quantum_error(error_gate2, ["cx"])

# noise_model = noise_bit_flip

# error = depolarizing_error(0.05, 1)
# noise_model.add_all_qubit_quantum_error(error, ['u1', 'u2', 'u3'])

# T1 = 221e3  # microseconds
# T2 = 131e3  # microseconds
# T1s = np.random.normal(T1, T1/5, L+1) # Microseconds.
# T2s = np.random.normal(T2, T2/5, L+1)  # Microseconds.

# # Truncate random T2s <= T1s
# T2s = np.array([min(T2s[j], 2 * T1s[j]) for j in range(L+1)])

# # Instruction times (in nanoseconds)
# time_u1 = 0   # virtual gate
# time_u2 = 50  # (single X90 pulse)
# time_u3 = 100 # (two X90 pulses)
# time_cx = 300
# time_reset = 1000  # 1 microsecond
# time_measure = 1000 # 1 microsecond

# # QuantumError objects
# errors_reset = [thermal_relaxation_error(t1, t2, time_reset)
#                 for t1, t2 in zip(T1s, T2s)]
# errors_measure = [thermal_relaxation_error(t1, t2, time_measure)
#                   for t1, t2 in zip(T1s, T2s)]
# errors_u1  = [thermal_relaxation_error(t1, t2, time_u1)
#               for t1, t2 in zip(T1s, T2s)]
# errors_u2  = [thermal_relaxation_error(t1, t2, time_u2)
#               for t1, t2 in zip(T1s, T2s)]
# errors_u3  = [thermal_relaxation_error(t1, t2, time_u3)
#               for t1, t2 in zip(T1s, T2s)]
# errors_cx = [[thermal_relaxation_error(t1a, t2a, time_cx).expand(
#              thermal_relaxation_error(t1b, t2b, time_cx))
#               for t1a, t2a in zip(T1s, T2s)]
#                for t1b, t2b in zip(T1s, T2s)]

# #noise_thermal = NoiseModel()
# for j in range(L+1):
#     noise_model.add_all_qubit_quantum_error(errors_reset[j], "reset", [j])
#     noise_model.add_all_qubit_quantum_error(errors_measure[j], "measure", [j])
#     noise_model.add_all_qubit_quantum_error(errors_u1[j], "u1", [j])
#     noise_model.add_all_qubit_quantum_error(errors_u2[j], "u2", [j])
#     noise_model.add_all_qubit_quantum_error(errors_u3[j], "u3", [j])
#     for k in range(L+1):
#         noise_model.add_all_qubit_quantum_error(errors_cx[j][k], "cx", [j, k])
#noise_model = noise_thermal

# [NOTE]: The noisy simulator requires we use Estimator() from qiskit_aer.primitives.

#noisy_sampler = Sampler(options=dict(backend_options=dict(noise_model=noise_model)))
# noisy_estimator = Estimator(options=dict(backend_options=dict(noise_model=noise_model)))

In [ ]:
def expectation_value_of_pauli_string_from_counts(pauli_string, counts_dictionary_from_measurement):
        """
        This function calculates the expectation value of a Pauli string from the counts dictionary
        obtained from the measurement.
        Parameters
        ----------
        pauli_string : str
            The Pauli string for which the expectation value is to be calculated.
        counts_dictionary_from_measurement : dict
                The counts dictionary obtained from the measurement.
        Returns
        -------
        float
            The expectation value of the Pauli string.

        """
        number_of_shots = sum(counts_dictionary_from_measurement.values())
        pauli_string_expectation_value = 0.0
        non_identity_indices = [i for i, char in enumerate(pauli_string) if char != "I"]
        for key, value in counts_dictionary_from_measurement.items():
                filtered_binary_string = "".join([char for i, char in enumerate(key) if i in non_identity_indices])
                pauli_string_expectation_value += ((-1)**sum(int(num) for num in filtered_binary_string))* (value/number_of_shots)
        return pauli_string_expectation_value

expectation_value_of_pauli_string_from_counts("XXYY", {"0000": 1000, "0001": 10, "0010": 10, "0111": 4})        

# Sampler [Returns shot counts for each observables]

In [ ]:
def current_expectation_value_sampler(initial_state_of_the_system,
                              hamiltonian_of_molecule,
                              LUMO_qubit_index,
                              operators_to_be_measured,
                              gamma_in,
                              gamma_out,
                              dt,
                              number_of_data_points,
                              gate_folding_number):

     """
     This function calculates the current expectation value of the I_in and I_out 
     operator as a function of time.
     Parameters
     ----------
     initial_state_of_the_system: str
          The initial state of the system.
     hamiltonian_of_molecule: SparsePauliOp
          The qubit Hamiltonian of the system.
     LUMO_qubit_index: int
          The index of the LUMO qubit counting from right to the left.          
     operators_to_be_measured: SparsePauliOp
          Operators to calculate expectation value.
     gamma_in: float
          The rate of the current in operator.
     gamma_out: float
          The rate of the current out operator.
     dt: float
          The time step.
     number_of_data_points: int
          The number of data points.
     gate_folding_number: int
          The number of times the local gate folding is performed.

     Returns
     -------
     I_in_current_as_a_function_of_time: list
          The current in operator as a function of time.
     I_out_current_as_a_function_of_time: list
          The current out operator as a function of time.
     """

     initial_state = Statevector.from_label(initial_state_of_the_system)

     counts_lst = []

     for ii in range(1, number_of_data_points):

          t_final = np.around(ii*dt,2)
          print("Time = ", t_final)
     
          # This section sets up the time evolution problem and the trotter circuit using the custom trotter class.
          problem = TimeEvolutionProblem(hamiltonian_of_molecule, initial_state = initial_state, time = t_final)
          trotterop = trotter.TrotterQRTE(num_timesteps = ii, LUMO_qubit_index = LUMO_qubit_index, gamma_out = gamma_out, gamma_in = gamma_in)
          result = trotterop.evolve(problem)
          trotter_circuit = result.evolved_state.decompose()
     
          for pauli_string_to_be_measured in operators_to_be_measured:

               trotter_circuit_temp = trotter_circuit.copy()

               # Reversing the Pauli string to match the qubit order in the circuit.
               pauli_string_to_be_measured_reversed = pauli_string_to_be_measured[::-1]
               for q in range(L):
                    pauli_matrix = pauli_string_to_be_measured_reversed[q]
                    if pauli_matrix == "X":
                         trotter_circuit_temp.h(q+1)
                    elif pauli_matrix == "Y":
                         trotter_circuit_temp.sdg(q+1)
                         trotter_circuit_temp.h(q+1)
                    elif pauli_matrix == "Z":
                         pass
                    elif pauli_matrix == "I":
                         pass            
                    
               if gate_folding_number != 0:
                    trotter_circuit_temp= local_gate_folding(trotter_circuit_temp, gate_folding_number)
               else:
                    trotter_circuit_temp = trotter_circuit_temp  

               # Measure all qubits excluding the ancilla qubit. 
               # Initialize a quantum register with L+1 qubits and a classical register with L bits
               qr = QuantumRegister(L+1, "q")
               cr = ClassicalRegister(L, "c")
               qc = QuantumCircuit(qr, cr)      

               for instr, qargs, cargs in trotter_circuit_temp:
                    qc.append(instr, qargs, cargs)

               # Measure qubits 1 to L and store the results in classical bits 0 to L-1.
               qc.barrier()
               for p in range(1, L+1):
                    qc.measure(qr[p], cr[p-1])     

               # Noiseless sampler.
               pm = generate_preset_pass_manager(optimization_level = 3)
               isa_circuit = pm.run(qc)
               isa_circuit = transpile(isa_circuit, backend = real_backend_simulator)
               job = sampler.run([isa_circuit], shots = 50000)
               result = job.result()
               pub_result = result[0]
               counts = pub_result.data.c.get_counts()

               # # Noisy sampler. 
               # pass_manager = generate_preset_pass_manager(3, AerSimulator())
               # isa_circuit = pass_manager.run(qc)
               # job = noisy_sampler.run([isa_circuit], shots = 50000)
               # result = job.result()
               # pub_result = result[0]
               # counts = pub_result.data.c.get_counts()

               folder_name = "simulator_data/reactant/2.0/2.0/"
               #folder_name = "simulator_data/transition_state_1/different_decay/2.0/1.e-2/"
               #folder_name = "simulator_data/transition_state_2/different_decay/2.0/1.e-2/"
               #folder_name = "simulator_data/product/different_decay/2.0/1.e-2"

               relative_path = os.path.join(folder_name, "counts_Allox_DNA_cutoff_threshold=" + str(cutoff_threshold) + "_" + str(pauli_string_to_be_measured) + "_t_final=" + str(t_final) + ".pickle")
               with open(relative_path, "wb") as f:
                    pickle.dump(counts, f)
               counts_lst.append(counts)

               print("Final depth of the trotter circuit = ", isa_circuit.depth())  
          
     return counts_lst
     # return None

# Estimator [Returns expectation value and their errors for each observabels]

In [ ]:
""" 
We mostly used Sampler in our research since it can be directly compared to the QPU data.
"""
def current_expectation_value_estimator(initial_state_of_the_system,
                              hamiltonian_of_molecule,
                              LUMO_qubit_index,
                              gamma_in,
                              gamma_out,
                              dt,
                              number_of_data_points):

     initial_state = Statevector.from_label(initial_state_of_the_system)

     circuit_lst = []
     statevector_lst = []
     expectation_values_mean_lst = []
     expectation_values_std_lst = []
     
     new_pauli_labels = [pauli + "I" for pauli in hamiltonian_of_molecule.paulis.to_labels()]
     qubit_hamiltonian_truncated_obs = SparsePauliOp(new_pauli_labels, hamiltonian_of_molecule.coeffs)

     for ii in range(1, number_of_data_points):

          t_final = np.around(ii*dt,2)
          print("Time = ", t_final)
          
          I_in_expectation_values_at_time_t_final = []
          I_out_expectation_values_at_time_t_final = []

          estimator = Estimator(mode=real_backend_simulator)

          # This section sets up the time evolution problem and the trotter circuit using the custom trotter class.
          problem = TimeEvolutionProblem(hamiltonian_of_molecule, initial_state = initial_state, time = t_final)

          # Noiseless.
          trotterop = trotter.TrotterQRTE(num_timesteps = ii, LUMO_qubit_index = LUMO_qubit_index, gamma_out = gamma_out, gamma_in = gamma_in, estimator = estimator)
          # Noisy.
          #trotterop = trotter.TrotterQRTE(num_timesteps = ii, LUMO_qubit_index = LUMO_qubit_index, gamma_out = gamma_out, gamma_in = gamma_in, estimator = noisy_estimator)

          result = trotterop.evolve(problem)
          trotter_circuit = result.evolved_state.decompose()
          circuit_lst.append(trotter_circuit)
          statevector_lst.append(Statevector(result.evolved_state))

          # Noiseless.
          pm = generate_preset_pass_manager(backend=real_backend_simulator, optimization_level=2)
          # Noisy.
          #pm = generate_preset_pass_manager(3, AerSimulator()) # Noisy simulator.

          isa_psi = pm.run(trotter_circuit)
          print("Final depth = ", isa_psi.depth())  
          isa_observables = qubit_hamiltonian_truncated_obs.apply_layout(isa_psi.layout)

          job = estimator.run([(isa_psi, isa_observables)])
          pub_result = job.result()[0]
          expectation_values_mean_lst.append(pub_result.data.evs)
          expectation_values_std_lst.append(pub_result.data.stds)
          
     return expectation_values_mean_lst, expectation_values_std_lst, statevector_lst, circuit_lst

#  Run Sampler

In [ ]:
qubit_hamiltonian_truncated

In [ ]:
""" 
In this section we list the sets of Pauli strings in the Hamiltonian that commutes with each other. This
enables us to measure only a small subset of observables on the QPU and then calculate the expectation value
of all the strings in each commuting group. We only need to do it once.
"""
qubit_hamiltonian_truncated_commuting_group = qubit_hamiltonian_truncated.group_commuting(qubit_wise = True)
qubit_hamiltonian_truncated_commuting_group[0]

In [ ]:
""" 
The following are the list of Pauli strings that are measured on the QPU for each Hamiltonian.

"""

# Allox in solution.
# Reactant.
sampler_measured_operator_lst = ["ZZYYZZYY","IIXXIIYY", "IIYYIIXX", "IIXXIIXX", "ZYZYIYZY", "IXZXIYZY", "IYZYIXZX", "IXZXIXZX", "ZZZZZZZZ"]
# Transition state 1.
# sampler_measured_operator_lst = ["ZYYYZYYY", "ZXXXZYYZ", "ZYYZZXXX", "IXXZIXXZ", "YZYZYZYZ",
#                                   "XZXIYZYI", "YZYIXZXI", "XZXIXZXI", "YZYZYZZY", "XZXIXZZX",
#                                   "YZZYYZYZ", "XZZXXZXI", "ZZZZZZZZ"]
# Transition state 2.
# sampler_measured_operator_lst = ["ZYYYZIYY", "IXXXIYYY", "IYYYIXXX", "IXXXIXXX" "ZZZZZZZZ"]  
# Product.
# sampler_measured_operator_lst = ["YZZYYZZY", "XZZXYZZY", "YZZYXZZX", "XZZXXZZX", "YZYYYZZY",
#                                  "XZXXYZZY", "YZYZXZZX", "XZXIXZZX", "YZZYYZYY", "XZZXYZYZ",
#                                  "YZZYXZXX", "XZZXXZXI", "YZYYYZYY", "XZXXYZYY", "YZYYXZXX",
#                                  "XZXXXZXX", "YYYZYYYZ", "XXXIYYZZ", "YYZZXXXI", "XXIIXXII",
#                                  "ZZZZYZZY", "ZIIIXZZX", "YZZYZZZZ", "XZZXZIII", "ZIIIYZYY",
#                                  "ZIIIXZXX", "YZYYZIII", "XZXXZIII", "ZIIIYYIZ", "ZIIIXXII",
#                                  "YYIZZIII", "XXIIZIII", "ZIIIZIII"]  
# 

# Allox in vacuum.
# # Alox-DNA in vacuum.
# Reactant.
# sampler_measured_operator_lst = ["YZYZYZYZ", "XZXIYZYI", "YZYIXZXI", "XZXIXZXI", "ZZZZZZZZ"]
# Product.
# sampler_measured_operator_lst = ["YZZYYZZY", "XZZXYZZY", "YZZYXZZX", "XZZXXZZX", "YYZYYYZY", "XXZXYYZY", "YYZYXXZX",
#                                    "XXZXXXZX", "ZZYYYZZY","ZIXXXZZX", "YZZYZZYY", "XZZXZIXX", "ZZYYZZZZ", "IIXXIIIZ", 
#                                    "ZZZZZZYY", "IIIZIIXX", "IIZZIIZZ"]                               

In [ ]:
gamma_in = 2.0
gamma_out = 2.0

dt = 0.05
number_of_data_points = 10 # Number of data points in the time evolution.

gate_folding_number = 0 # Number of times the local gate folding is performed for zero noise extrapolation.

# Initial state of the system. We start with the lowest half-filled state in each case.
state = "00110011" # Reactant.
#state = "11110000" # Transition state 1.
#state = "00110011" # Transition state 2.
# state = "00110011" # Product.

current_expectation_value_sampler = current_expectation_value_sampler(state, qubit_hamiltonian_truncated, LUMO_index, sampler_measured_operator_lst, gamma_in, gamma_out, dt, number_of_data_points, gate_folding_number)

# Run Estimator

In [ ]:
# gamma_in = 2.0
# gamma_out = 1.e-2
# dt = 0.05
# number_of_data_points = 10
# gate_folding_number = 0
# #initial_state = "00010111" # Reactant # Es = 00110101.
# #initial_state = "11110000" # Transition state 1.
# #initial_state = "00110011" # Transition state 2 Es = 00010111 or 01110001.
# initial_state = "00110011" # Product.
# #current_mean, current_std, state_lst, circuit_lst = current_expectation_value_estimator(initial_state, qubit_hamiltonian_truncated, LUMO_index, gamma_in, gamma_out, dt, number_of_data_points)

In [ ]:
"""
In this section we plot the energy as a function of time. To obtain error bars on the energy, we run the same
post-processing. We first divide the counts data into a number sublists (16 in this code) and then calculate the mean
and standard deviation of the energy. This step is necessary only to obtain the error bars on the energy and can be
skipped if error bars are not required.

"""

import random
from collections import Counter

# This function takes the counts data and returns a set of lists made up from the counts data.

def divide_into_sublists(lst, number_of_sublists):
    # Ensure the list length is divisible by N
    assert len(lst) % number_of_sublists == 0, "List length must be divisible by number_of_sublists"
    # Calculate the size of each sublist
    sublist_size = len(lst) // number_of_sublists
    # Divide the list into N sublists
    sublists = [lst[i * sublist_size:(i + 1) * sublist_size] for i in range(number_of_sublists)]
    return sublists

def expectation_value_of_pauli_string_from_counts(pauli_string, counts_dictionary_from_measurement):
        """
        This function calculates the expectation value of a Pauli string from the counts dictionary
        obtained from the measurement.
        Parameters
        ----------
        pauli_string : str
            The Pauli string for which the expectation value is to be calculated.
        counts_dictionary_from_measurement : dict
                The counts dictionary obtained from the measurement.
        Returns
        -------
        float
            The expectation value of the Pauli string.

        """
        number_of_shots = sum(counts_dictionary_from_measurement.values())
        pauli_string_expectation_value = 0.0
        non_identity_indices = [i for i, char in enumerate(pauli_string) if char != "I"]
        for key, value in counts_dictionary_from_measurement.items():
                filtered_binary_string = "".join([char for i, char in enumerate(key) if i in non_identity_indices])
                pauli_string_expectation_value += ((-1)**sum(int(num) for num in filtered_binary_string))* (value/number_of_shots)
        return pauli_string_expectation_value    

In [ ]:
folder_name = "simulator_data/reactant/2.0/2.0/"
# folder_name = "simulator_data/reactant/2.0/1.e-2/"
# folder_name = "simulator_data/reactant/2.0/low"
 
#folder_name = "simulator_data/reactant/vacuum/2.0/1.e-2"
#folder_name = "simulator_data/reactant/vacuum/2.0/low"

# folder_name = "simulator_data/transition_state_1/2.0/2.0/"

#folder_name = "simulator_data/product/2.0/1.e-2/"
# folder_name = "simulator_data/product/2.0/1.e-2/"
# folder_name = "simulator_data/product/2.0/low/"
#folder_name = "simulator_data/product/vacuum/2.0/low"

energy_as_function_of_time_lst_sim = []
time_lst = np.array([np.around(t*dt,2) for t in range(1,number_of_data_points)])
for time in time_lst:
     # Iterating over each commuting group.
     energy_at_time_t = 0.0
     for group_index in range(len(qubit_hamiltonian_truncated_commuting_group)):

          group_paulis = qubit_hamiltonian_truncated_commuting_group[group_index].paulis.to_labels()
          group_coeffs = qubit_hamiltonian_truncated_commuting_group[group_index].coeffs

          # Per group there is one measured pauli string.
          measured_pauli_string = sampler_measured_operator_lst[group_index]

          relative_path = os.path.join(folder_name,"counts_Allox_DNA_cutoff_threshold="+str(cutoff_threshold)+"_"+str(measured_pauli_string)+"_t_final="+str(time)+".pickle")
          with open(relative_path, "rb") as f:
               counts_measured_sampler = pickle.load(f)     

          for jj in range(len(group_paulis)):
               pauli_string = group_paulis[jj]
               pauli_coeff = group_coeffs[jj]
               energy_at_time_t += pauli_coeff*expectation_value_of_pauli_string_from_counts(pauli_string, counts_measured_sampler)
     energy_as_function_of_time_lst_sim.append(energy_at_time_t)  

In [ ]:
"""
In this section we plot the energy as a function of time. To obtain error bars on the energy, we run the same
post-processing. We first divide the counts data into a number sublists (16 in this code) and then calculate the mean
and standard deviation of the energy. This step is necessary only to obtain the error bars on the energy and can be
skipped if error bars are not required.

"""

import random
from collections import Counter

# This function takes the counts data and returns a set of lists made up from the counts data.

def divide_into_sublists(lst, number_of_sublists):
    # Ensure the list length is divisible by N
    assert len(lst) % number_of_sublists == 0, "List length must be divisible by number_of_sublists"
    # Calculate the size of each sublist
    sublist_size = len(lst) // number_of_sublists
    # Divide the list into N sublists
    sublists = [lst[i * sublist_size:(i + 1) * sublist_size] for i in range(number_of_sublists)]
    return sublists

def expectation_value_of_pauli_string_from_counts(pauli_string, counts_dictionary_from_measurement):
        """
        This function calculates the expectation value of a Pauli string from the counts dictionary
        obtained from the measurement.
        Parameters
        ----------
        pauli_string : str
            The Pauli string for which the expectation value is to be calculated.
        counts_dictionary_from_measurement : dict
                The counts dictionary obtained from the measurement.
        Returns
        -------
        float
            The expectation value of the Pauli string.

        """
        number_of_shots = sum(counts_dictionary_from_measurement.values())
        pauli_string_expectation_value = 0.0
        non_identity_indices = [i for i, char in enumerate(pauli_string) if char != "I"]
        for key, value in counts_dictionary_from_measurement.items():
                filtered_binary_string = "".join([char for i, char in enumerate(key) if i in non_identity_indices])
                pauli_string_expectation_value += ((-1)**sum(int(num) for num in filtered_binary_string))* (value/number_of_shots)
        return pauli_string_expectation_value    

# The following are the folders where the raw data is stored.
folder_name = "real_device_data/reactant/2.0/2.0/"   # gamma_in = 2.0, gamma_out = 2.0
# folder_name = "real_device_data/reactant/2.0/1.e-2/" # gamma_in = 1.e-2, gamma_out = 2.0
# folder_name = "real_device_data/reactant/2.0/low"    # gamma_in = 2.0, gamma_out = 1.e-2

# folder_name = "real_device_data/transition_state_1/2.0/2.0/"   # gamma_in = 2.0, gamma_out = 2.0
# folder_name = "real_device_data/transition_state_1/2.0/1.e-2/" # gamma_in = 1.e-2, gamma_out = 2.0
# folder_name = "real_device_data/transition_state_1/2.0/low"    # gamma_in = 2.0, gamma_out = 1.e-2

# folder_name = "real_device_data/transition_state_2/2.0/2.0/"   # gamma_in = 2.0, gamma_out = 2.0
# folder_name = "real_device_data/transition_state_2/2.0/1.e-2/" # gamma_in = 1.e-2, gamma_out = 2.0
# folder_name = "real_device_data/transition_state_2/2.0/low"    # gamma_in = 2.0, gamma_out = 1.e-2

# folder_name = "real_device_data/product/2.0/2.0/"   # gamma_in = 2.0, gamma_out = 2.0
# folder_name = "real_device_data/product/2.0/1.e-2/" # gamma_in = 1.e-2, gamma_out = 2.0
# folder_name = "real_device_data/product/2.0/low"    # gamma_in = 2.0, gamma_out = 1.e-2

time_lst = np.array([np.around(t*dt,2) for t in range(1,number_of_data_points)])

energy_as_function_of_time_lst_mean = []
energy_as_function_of_time_lst_seom = []

for time in time_lst:

    energy_at_time_t_mean = 0.0
    energy_at_time_t_seom = 0.0

    # Iterating over each commuting group.
    for group_index in range(len(qubit_hamiltonian_truncated_commuting_group)):

        group_paulis = qubit_hamiltonian_truncated_commuting_group[group_index].paulis.to_labels()
        group_coeffs = qubit_hamiltonian_truncated_commuting_group[group_index].coeffs

        # Per group there is one measured pauli string.
        measured_pauli_string = sampler_measured_operator_lst[group_index]

        relative_path = os.path.join(folder_name,"counts_Allox_DNA_cutoff_threshold="+str(cutoff_threshold)+"_"+str(measured_pauli_string)+"_t_final="+str(time)+".pickle")
        with open(relative_path, "rb") as f:
            counts_measured_sampler = pickle.load(f)     

        number_of_sublists = 16
        counts_measured_sampler_lst = [key for key, value in counts_measured_sampler.items() for _ in range(value)]
        random.shuffle(counts_measured_sampler_lst)
        counts_measured_sampler_sublists = divide_into_sublists(counts_measured_sampler_lst, number_of_sublists)
        counts_measured_sampler = [dict(Counter(counts_measured_sampler_sublists[i])) for i in range(number_of_sublists)]

        energy_per_sublist_lst = []
        for counts_sublist in counts_measured_sampler:
            energy_at_time_t_per_sublist = 0.0
            for jj in range(len(group_paulis)):
                pauli_string = group_paulis[jj]
                pauli_coeff = group_coeffs[jj]
                energy_at_time_t_per_sublist += pauli_coeff*expectation_value_of_pauli_string_from_counts(pauli_string, counts_sublist)
            energy_per_sublist_lst.append(energy_at_time_t_per_sublist)
        energy_at_time_t_mean += np.mean(energy_per_sublist_lst)
        energy_at_time_t_seom += np.std(energy_per_sublist_lst)/np.sqrt(number_of_sublists-1)

    energy_as_function_of_time_lst_mean.append(energy_at_time_t_mean)
    energy_as_function_of_time_lst_seom.append(energy_at_time_t_seom)

energy_as_function_of_time_lst_mean_real_device = np.array(energy_as_function_of_time_lst_mean)#+ (-0.6137996950173608)
energy_as_function_of_time_lst_seom_real_device = np.array(energy_as_function_of_time_lst_seom)

In [ ]:
"""  
This section is optional. It is used to shift the energy values to the desired reference energy by subtracting the identity
term in each Hamiltonian. The qualitative behavior of the energy current as a function of time remains the same. We also performed
additional energy shift that occurs due to the truncation of the Hamiltonian using active space transformation and
the cutoff threshold. The data is obtained from Milestone 3 from the plot of QM/MM energy as a function of reaction coordinate.
That energy shift is not included in this code.
"""

# These are the list of identity terms for each Hamiltonian in solution and vacuum.
# reactant_identity_term_sol = -2.77543916
# reactant_identity_term_vac = -2.52473403
# transition_state_1_identity_term_sol = -5.01201892 
# transition_state_2_identity_term_sol = -2.91638423
# product_identity_term_sol = -5.1763464 
#product_identity_term_vac = -4.60200667

# energy_as_function_of_time_lst_sim = np.array(energy_as_function_of_time_lst_sim)
# Change the follwing to the desired identity term.
# energy_as_function_of_time_lst_mean_sim_shifted  = energy_as_function_of_time_lst_sim - reactant_identity_term_sol
# energy_as_function_of_time_lst_mean_sim_shifted  = energy_as_function_of_time_lst_sim - reactant_identity_term_vac

# energy_as_function_of_time_lst_mean_sim_shifted = energy_as_function_of_time_lst_sim - transition_state_1_identity_term_sol

# energy_as_function_of_time_lst_mean_sim_shifted = energy_as_function_of_time_lst_sim - transition_state_2_identity_term_sol

# energy_as_function_of_time_lst_mean_sim_shifted  = energy_as_function_of_time_lst_sim - product_identity_term_sol
# energy_as_function_of_time_lst_mean_sim_shifted  = energy_as_function_of_time_lst_sim - product_identity_term_vac

In [ ]:
fig, axs = plt.subplots(1, 1, figsize=(10, 6))
axs.errorbar(time_lst, energy_as_function_of_time_lst_mean_real_device.real, yerr = energy_as_function_of_time_lst_seom_real_device.real, fmt = "o", color = "tab:red", capsize = 5, elinewidth = 2, markeredgewidth = 2, ms = 10, label = "Real Device Sampler data")
axs.plot(time_lst, np.real(energy_as_function_of_time_lst_sim), "o-", label = "Noisy simulator", linewidth = 2, markersize = 10)
plt.title(r"Reactant $\gamma_{in} = "+str(gamma_in)+", \gamma_{out} = "+str(gamma_out)+"$")

plt.xlabel("Time")
plt.ylabel("Energy")
axs.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
#plt.ylim(-1.2, -0.4)
for spine in axs.spines.values():
    spine.set_linewidth(2)
axs.tick_params(axis="x", direction="inout", length=10, width=2, color="k")
axs.tick_params(axis="y", direction="inout", length=10, width=2, color="k")
axs.xaxis.set_minor_locator(ticker.AutoMinorLocator())
axs.yaxis.set_minor_locator(ticker.AutoMinorLocator())
axs.tick_params(which="minor", length=5, width=1, direction='in')
# plt.savefig("reactant_gamma_in_2.0_gamma_out_2.0.png", dpi = 600, bbox_inches = "tight")
plt.show()